In [15]:
import duckdb
import pandas as pd

con = duckdb.connect('/mnt/disk2/qode_edw_bp.db')

exchange = 'NSE'
underlying = 'NIFTY'
expiry_old = '20230330'
expiry_new = '20230329'

tables = con.execute(f"""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'market_data'
      AND table_name LIKE '{exchange}_Options_{underlying}_{expiry_old}_%'
""").fetchdf()['table_name'].tolist()

print(tables)

for old_table in tables:
    parts = old_table.split('_')
    strike = parts[4]
    opt_type = parts[5]
    new_table = f"{exchange}_Options_{underlying}_{expiry_new}_{strike}_{opt_type}"

    df = con.execute(f"SELECT * FROM market_data.{old_table}").fetchdf()

    con.execute(f"""
        CREATE TABLE IF NOT EXISTS market_data.{new_table} AS
        SELECT * FROM market_data.{old_table} WHERE 1=0
    """)

    print(df.head())

    df_new = con.execute(f"SELECT * FROM market_data.{new_table}").fetchdf()

    print(df_new.head())

    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS iv")
    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS theta")
    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS rho")
    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS vega")
    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS delta")
    con.execute(f"ALTER TABLE market_data.{new_table} DROP COLUMN IF EXISTS gamma")
    
    col_order = con.execute(f"PRAGMA table_info(market_data.{new_table})").fetchdf()['name'].tolist()
    df = df[col_order]  # Reorder columns

    con.register('df_view', df)
    con.execute(f"INSERT INTO market_data.{new_table} SELECT * FROM df_view")
    con.unregister('df_view')

    print(f"Inserted {len(df)} rows from {old_table} to {new_table}")

con.close()

['NSE_Options_NIFTY_20230330_11000_put', 'NSE_Options_NIFTY_20230330_12000_put', 'NSE_Options_NIFTY_20230330_13000_call', 'NSE_Options_NIFTY_20230330_13000_put', 'NSE_Options_NIFTY_20230330_14000_call', 'NSE_Options_NIFTY_20230330_14000_put', 'NSE_Options_NIFTY_20230330_15000_call', 'NSE_Options_NIFTY_20230330_15000_put', 'NSE_Options_NIFTY_20230330_16000_call', 'NSE_Options_NIFTY_20230330_16000_put', 'NSE_Options_NIFTY_20230330_17000_call', 'NSE_Options_NIFTY_20230330_17000_put', 'NSE_Options_NIFTY_20230330_18000_call', 'NSE_Options_NIFTY_20230330_18000_put', 'NSE_Options_NIFTY_20230330_19000_call', 'NSE_Options_NIFTY_20230330_19000_put', 'NSE_Options_NIFTY_20230330_20000_call', 'NSE_Options_NIFTY_20230330_20000_put', 'NSE_Options_NIFTY_20230330_21000_call', 'NSE_Options_NIFTY_20230330_21000_put', 'NSE_Options_NIFTY_20230330_22000_call', 'NSE_Options_NIFTY_20230330_22000_put', 'NSE_Options_NIFTY_20230330_23000_call', 'NSE_Options_NIFTY_20230330_24000_call']
            timestamp     o

: 